# System Design: Real-Time Recommendation Engine

## 1. Architecture Overview
**Two-Tower Architecture**:
1. **Retrieval Layer**: Fast candidate generation (HNSW/Vector DB) -> Select top 100 from 1M.
2. **Ranking Layer**: Heavy model (Deep Learning) -> Sort top 100 for final 10 display.

In [ ]:
import numpy as np
from src.core.math_operations import cosine_similarity
from src.production.caching import LRUCache

# --- Component 1: Matrix Factorization for Embeddings ---
class EmbeddingLayer:
    def __init__(self, num_items, dim=64):
        self.embeddings = np.random.randn(num_items, dim)
        
    def get_embedding(self, item_id):
        return self.embeddings[item_id]

# --- Component 2: Vector Search (Simplified HNSW) ---
class VectorSearch:
    def __init__(self, item_embeddings):
        self.index = item_embeddings
        
    def search(self, query_vec, k=5):
        # Brute force for demo (use HNSW in prod)
        scores = []
        for i, vec in enumerate(self.index):
            score = cosine_similarity(query_vec, vec)
            scores.append((i, score))
        
        scores.sort(key=lambda x: x[1], reverse=True)
        return scores[:k]

# --- Component 3: Ranking Service with Caching ---
cache = LRUCache(max_size=1000)
emb_layer = EmbeddingLayer(num_items=1000)
search_engine = VectorSearch(emb_layer.embeddings)

def get_recommendations(user_vec):
    # 1. Check Cache (Caching user-specific result lists is complex, usually cache items)
    # Here we mock user_vec hash as key
    cache_key = str(user_vec[:3]) 
    cached_res = cache.get(cache_key)
    if cached_res:
        return cached_res, True
    
    # 2. Retrieve Candidates
    candidates = search_engine.search(user_vec, k=10)
    
    # 3. Re-Rank (Mock logic: just use similarity)
    final_recs = [c[0] for c in candidates]
    
    # 4. Update Cache
    cache.set(cache_key, final_recs)
    
    return final_recs, False

# Simulate User
user_query_vector = np.random.randn(64)

print("Request 1:", get_recommendations(user_query_vector))
print("Request 2 (Cached):", get_recommendations(user_query_vector))